<a href="https://colab.research.google.com/github/deekshita008/BranchCode_99_Team1/blob/main/%F0%9F%97%91%F0%9F%97%91Waste_Segregation_using_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'waste-segregation-image-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2495983%2F4235079%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240527%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240527T171354Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6a79de68ea34a05f1f5e120969ffb6e8822c356cf29bda0c0344447866fba41594b5591267fcec8026aee44d5bf1973880448040b8f6ba7332c3a570adf28a275dcd0004b22435bdb33e460031a4f6f42fac3e7ca096e4eec48abd802697368f4bbc5a957af22bcc0901022032b106864cba49710f8ce3359139ed859350c1a1e702f2104f52bdd155d17799a1d6df2b2b313467ef9b7d854e12872531ce1fc4d170d68a2854c02f911ca9f847f00ee7ace797c9cd8e68fd1be9a6cd51b81968149ba82bb7dbb3c93153a847635c32a1d8f5ae9e88f6ee314c451ae8c38b2d945b698677e7316acf3f40427c25928300e52ca01e5b3d4ed7b56cc8b83c597c19'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1253362034 bytes downloaded
Downloaded and uncompressed: waste-segregation-image-dataset
Data source import complete.


In [3]:
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# Check for CPU/GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_transform = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(20),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
train_data = datasets.ImageFolder(root="../input/waste-segregation-image-dataset/Dataset/train", transform=train_transform)
trainDataLoader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)


test_transform = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
test_data = datasets.ImageFolder(root="../input/waste-segregation-image-dataset/Dataset/val", transform=test_transform)
testDataLoader = DataLoader(dataset=test_data, batch_size=32, shuffle=True)

# Load Pre-trained model
model = models.resnet50(pretrained=True)

print("Before: \n", model, "\n\n")

# Freeze our Feature parameters of the model
# Model Parameters: Pre-Learned Weights & Biases
for param in model.parameters():
    # Don't allow gradients/features to update.
    param.requires_grad = False

# Replace the Classification layer with our custom classification layer
from collections import OrderedDict

# Define a sequence of custom layers
# Pass in a OrderedDict to name each of these layers and corresponding functions
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(2048, 512)),
    ('relu', nn.ReLU()),
    ('dropout', nn.Dropout(p=0.2)),
    ('fc2', nn.Linear(512, 2)),
    ('output', nn.LogSoftmax(dim=1))
]))

# Add custom layers to the pre-trained model
model.fc = classifier

print("After: \n", model, "\n")

# Train the model with newly added classifier layer
model.to(device)

# Epochs
epochs = 10

# Loss Criterion
criterion = nn.NLLLoss()

# Optimizer
# Since, we only need to update the parameters for the Model classifier.
# So, we use "model.classifier.parameters()" instead of "model.parameters()".
optimizer = optim.Adam(params=model.fc.parameters(), lr=0.001)

train_losses, test_losses = [], []

print("Starting Model Training...")

# ------------- Training Loop ------------
for e in range(epochs):
    running_loss = 0
    # ------------- Training Loop -----------
    for images, labels in trainDataLoader:
        images, labels = Variable(images), Variable(labels)
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model.forward(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # ------------ Validation Loop -----------
    else:
        test_loss = 0
        accuracy = 0
        with torch.no_grad():
            model.eval()
            for images, labels in testDataLoader:
                images, labels = Variable(images), Variable(labels)
                images, labels = images.to(device), labels.to(device)
                logps = model.forward(images)
                test_loss += criterion(logps, labels)
                # Actual Probability distribution from Log Probabilities
                ps = torch.exp(logps)
                # topk: returns K largest elements of the given input tensor along a given dimension
                top_k, top_class = ps.topk(1, dim=1)
                # Check if Predicted Output is equal to the Actual Labels
                equals = top_class == labels.view(*top_class.shape)
                # Calculate Accuracy on the Test Dataset
                accuracy += torch.mean(equals.type(torch.FloatTensor))

        # ** Trun back ON the Dropouts for Model Training **
        model.train()

        # Keep track of Training and Test Loss
        train_losses.append(running_loss / len(trainDataLoader))
        test_losses.append(test_loss / len(testDataLoader))

        print("Epoch: {}/{}".format(e + 1, epochs),
              "Training Loss: {:.3f}\t".format(running_loss / len(trainDataLoader)),
              "Test Loss: {:.3f}\t".format(test_loss / len(testDataLoader)),
              "Test Accuracy: {:.3f}\t".format(accuracy / len(testDataLoader)))

print("\nDone...")

Before: 
 ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), s

KeyboardInterrupt: 